In [23]:
import cobra
from cobra.io import read_sbml_model
import cobra.test
from mewpy.simulation import get_simulator
from cobamp.wrappers import KShortestEFMEnumeratorWrapper
from mewpy.simulation import get_simulator
from cobra.flux_analysis import flux_variability_analysis

In [10]:
model = read_sbml_model('./iML1515.xml')
model3 = model.copy()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.004565,0,0.00%
cl_e,EX_cl_e,0.004565,0,0.00%
cobalt2_e,EX_cobalt2_e,2.192E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006218,0,0.00%
fe2_e,EX_fe2_e,0.01409,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
k_e,EX_k_e,0.1712,0,0.00%
mg2_e,EX_mg2_e,0.007608,0,0.00%
mn2_e,EX_mn2_e,0.000606,0,0.00%
mobd_e,EX_mobd_e,6.139E-06,0,0.00%


In [11]:
#Implementar as restrições na cópia do modelo
model3.reactions.get_by_id("EX_o2_e").lower_bound = 0
model3.reactions.get_by_id("EX_glc__D_e").lower_bound = -15

In [4]:
model3.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.001385,0,0.00%
cl_e,EX_cl_e,0.001385,0,0.00%
co2_e,EX_co2_e,0.09771,1,0.11%
cobalt2_e,EX_cobalt2_e,6.652E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001887,0,0.00%
fe2_e,EX_fe2_e,0.002196,0,0.00%
fe3_e,EX_fe3_e,0.002078,0,0.00%
glc__D_e,EX_glc__D_e,15,6,99.89%
h2o_e,EX_h2o_e,5.865,0,0.00%
k_e,EX_k_e,0.05194,0,0.00%


Para perceber a melhor estratégia para a produção de acetato, primeiro é necessário saber as vias metabólicas que poderam interagir com a sintese do composto.

Usando o modelo Esher
https://escher.github.io/#/app?map=iJO1366.Central%20metabolism&tool=Builder&model=e_coli_core

Reparamos que muitas das reações que puderam influênciar a sintese de acetato são reações que envolvem a sinstese de acetil-coa, acetaldeido, etanol, formato, fosfoenolpiruvato, lactato e precursores do ciclo de Krebs.
Por isso, selecionamos reações que fossem relevantes, de modo a verificar o seu impacto na produção do composto de interesse. 

Reações como:
    -'ACALDtpp', 'CS', 'PFL', 'LDH_D', 'PPS', 'ACS', 'ALCD2x','ACALD','PPC', 'POX', 
     'ACALDtpp','H2tpp','FHL','FORtppi','FORtex','ETOHtrpp', 'L_LACD3'

In [5]:
#Verificar se as reações pertencem ao modelo
lst_reac = ["ACALDtpp",'ACALD', "CS", "PFL", "LDH_D", 
            "PPS",'PPC', "ACS", "ALCD2x", 'POX', 
            'ACALDtpp','H2tpp','FHL','FORtppi','FORtex',
           'ETOHtrpp', 'L_LACD3']

for j in lst_reac:
    ver = j in model3.reactions
    if ver == True:
        print(f'{j} pertence ao modelo!')
    else:
        print(f'{j} não pertence ao modelo!')
        lst_reac.remove(j)

ACALDtpp pertence ao modelo!
ACALD pertence ao modelo!
CS pertence ao modelo!
PFL pertence ao modelo!
LDH_D pertence ao modelo!
PPS pertence ao modelo!
PPC pertence ao modelo!
ACS pertence ao modelo!
ALCD2x pertence ao modelo!
POX pertence ao modelo!
ACALDtpp pertence ao modelo!
H2tpp pertence ao modelo!
FHL pertence ao modelo!
FORtppi pertence ao modelo!
FORtex pertence ao modelo!
ETOHtrpp pertence ao modelo!
L_LACD3 pertence ao modelo!


In [6]:
simul = get_simulator(model3) #Necessário para ver as reações essenciais

In [8]:
#verificar se as reações são essenciais para o funcionamento do modelo
for i in lst_reac:
    sera = i in simul.essential_reactions
    if sera == True:
        print(f'{i} pertence as reações essenciais!')
        lst_reac.remove(i)
    else:
        print(f'{i} não pertence as reações essenciais.')

ACALDtpp não pertence as reações essenciais.
ACALD não pertence as reações essenciais.
CS pertence as reações essenciais!
LDH_D não pertence as reações essenciais.
PPS não pertence as reações essenciais.
PPC não pertence as reações essenciais.
ACS não pertence as reações essenciais.
ALCD2x não pertence as reações essenciais.
POX não pertence as reações essenciais.
ACALDtpp não pertence as reações essenciais.
H2tpp não pertence as reações essenciais.
FHL não pertence as reações essenciais.
FORtppi não pertence as reações essenciais.
FORtex não pertence as reações essenciais.
ETOHtrpp não pertence as reações essenciais.
L_LACD3 não pertence as reações essenciais.


Dos intervenientes só a reação da citrato sintetase ("CS") é essencial, já as outras poderam ser alteradas.

In [41]:
#Para perceber as alterações no fluxo do acetato e de biomassa 
    #regista-se o fluxo no WT
model_mut = model3.copy()
loop_reactions = [model_mut.reactions.EX_ac_e,model_mut.reactions.BIOMASS_Ec_iML1515_core_75p37M]
print(flux_variability_analysis(model_mut, reaction_list=loop_reactions, loopless=False))

                                  minimum    maximum
EX_ac_e                         13.025230  13.025230
BIOMASS_Ec_iML1515_core_75p37M   0.266098   0.266098


In [42]:
#Registar o fluxo das reações de interesse em mutantes
for reaccao in lst_reac:
    with model_mut:
        model_mut.reactions.get_by_id(reaccao).upper_bound=0
        model_mut.reactions.get_by_id(reaccao).lower_bound=0
        print(f'-Fluxo das reações de interesse em mutantes sem a reação {reaccao}')
        print(flux_variability_analysis(model_mut, reaction_list=loop_reactions, loopless=False))
        print(' ')

-Fluxo das reações de interesse em mutantes sem a reação ACALDtpp
                                  minimum    maximum
EX_ac_e                         13.025230  13.025230
BIOMASS_Ec_iML1515_core_75p37M   0.266098   0.266098
 
-Fluxo das reações de interesse em mutantes sem a reação ACALD
                                 minimum    maximum
EX_ac_e                         0.112451  26.797959
BIOMASS_Ec_iML1515_core_75p37M  0.208160   0.208160
 
-Fluxo das reações de interesse em mutantes sem a reação PFL
                                 minimum       maximum
EX_ac_e                         0.000000 -2.222209e-13
BIOMASS_Ec_iML1515_core_75p37M  0.204068  2.040675e-01
 
-Fluxo das reações de interesse em mutantes sem a reação LDH_D
                                  minimum    maximum
EX_ac_e                         13.025233  13.025291
BIOMASS_Ec_iML1515_core_75p37M   0.266098   0.266098
 
-Fluxo das reações de interesse em mutantes sem a reação PPS
                                  minim

Em muitos dos mutantes não ocorre a alteração do fluxo da síntese de acetato extracelular. Contudo, as vias que levam à sintese de formato ('PFL','FORtppi','FORtex') parecem ter grande influência na sintese de acetato em condições anaeróbias. 

Outros mutantes que não sintetizam etanol ou acetaldeído ('ETOHtrpp','ACALD','ALCD2x') em condições anaeróbias mostraram também ter influência na sintese de acetato, onde se registou o maior fluxo possivel da reação EX_ac_e.

___________________________________________________________________________

In [31]:
#Não aplicando condições ambientais...
with model3:
        model3.reactions.get_by_id('EX_o2_e').lower_bound=-10
        
        simul = get_simulator(model3)
        result = simul.simulate(method='FBA')
        
        print(model3.summary())

Objective
1.0 BIOMASS_Ec_iML1515_core_75p37M = 0.6577963639344184

Uptake
------
Metabolite      Reaction      Flux  C-Number   C-Flux
     ca2_e      EX_ca2_e  0.003424         0    0.00%
      cl_e       EX_cl_e  0.003424         0    0.00%
 cobalt2_e  EX_cobalt2_e 1.644E-05         0    0.00%
     cu2_e      EX_cu2_e 0.0004664         0    0.00%
     fe2_e      EX_fe2_e  0.005429         0    0.00%
     fe3_e      EX_fe3_e  0.005136         0    0.00%
  glc__D_e   EX_glc__D_e        15         6  100.00%
       k_e        EX_k_e    0.1284         0    0.00%
     mg2_e      EX_mg2_e  0.005706         0    0.00%
     mn2_e      EX_mn2_e 0.0004545         0    0.00%
    mobd_e     EX_mobd_e 4.605E-06         0    0.00%
     nh4_e      EX_nh4_e     7.104         0    0.00%
     ni2_e      EX_ni2_e 0.0002125         0    0.00%
      o2_e       EX_o2_e        10         0    0.00%
      pi_e       EX_pi_e    0.6345         0    0.00%
     so4_e      EX_so4_e    0.1656         0    0.00%
 

Se não forem aplicadas condições anaeróbias, o modelo de E.coli é capaz de produzir mais acetato e com uma produção de biomassa mais benéfica.